In [1]:
#! pip install ipywidgets
#! jupyter nbextension enable --py widgetsnbextension --sys-prefix
from ipywidgets import interact
from ipywidgets.widgets import FloatText, BoundedIntText, FloatSlider, IntSlider, IntRangeSlider, fixed, Text

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Widgets for interactive functions / plots
style = {'description_width': 'initial'}

debt_text = FloatText(value=100000, step=100, description='Debt:')
save_text = FloatText(value=100000, step=100, description='Amount to save:', style=style)
monthly_save_text = FloatText(value=100000, step=100, description='Monthly saving:', style=style)

payoff_slider = FloatSlider(value=1000, step=10, min=500, max=10_000, description='Monthly pay off:', style=style)
rent_slider = FloatSlider(value=1000, step=10, min=500, max=10_000, description='Monthly rent:', style=style)

utilities_slider = FloatSlider(value=800/2300, step=0.01, min=0, max=1, description='% of rent as utilities:', style=style)

interest_slider = FloatSlider(value=0.15, step=0.01, min=0, max=0.25, description='Interest:')
interest_text = Text(value='0.05, 0.05, 0.05, 0.05, 0.05, 0.05', description="Interests")

tax_slider = FloatSlider(value=0.19, step=0.01, min=0, max=0.30, description='Tax:')

months_slider = IntSlider(value=1, step=1, min=1, max=360, description='Months:')
years_range_slider = IntRangeSlider(value=(1, 40), step=1, min=1, max=100, description='Years:')

# Loans

### Mortgage

We know already that if we have initial deposit $D_0$ and different contribution per compound $c$ then the general formula for $D_n$:
\begin{equation}
$D_n = D_0 q^{nm} + c \sum_{i=0}^{nm-1} q^i = D_0 q^{nm} + c \cdot \frac{1 - q^{nm}}{1-q}$.
\end{equation}

Deposits and debts are actually the same thing -- there are just different sides of contract. In deposit you loan money to the bank and in return get some interests, just like with debt. This is why we can use formulas from deposits, but we need to change initial deposit to be negative, as to represent our debt and contributions will be added to negative amount so that we are getting closer to paying it of.

In [3]:
def debt(debt: float, n: int, payoff: float, interest: float, tax: float = 0.19):
    """Calculate debt amount after n compound periods."""
    r = interest / 12 * (1-tax)
    d_n = (debt * (1 + r) ** n + payoff * (1 - (1+r)**n) / (-r))
    return d_n

In [4]:
def debt_test(debt: float, n: int, payoff: float, interest: float, tax: float = 0.19):
    """Compute debt amount after n compound periods."""
    d_n = debt
    r = interest / 12 * (1-tax)
    for i in range(n):
        d_n = d_n * (1+r) + payoff
        print(f'period={i+1} with payoff, debt={d_n:.2f}')
    return d_n

#d1 = debt(debt=-100_000, n=120, payoff=1443.39, interest=0.15)
#d2 = debt_test(debt=-100_000, n=120, payoff=1443.39, interest=0.15)

## Amount of payoff
We are interested in the amount of payoff per period $c$ given initial debt $D_0$, $p\%$ interest rate, $s\%$ tax fee and $m=12$ frequency of compounding. Let's denote $q = 1 +r, \; r = \frac{p}{m} \cdot (1-s)$ To get formula for it, we need to know the number of periods $N=nm$ we want to pay off debt. Paying off the debt in $N$ periods mean we have equation $D_n = 0$:

\begin{align}
    D_0 q^{nm} + c \cdot \frac{1 - q^N}{1-q} = 0 \\
    c = \frac{-D_0 q^N (1-q)}{1-q^N}
\end{align}




In [5]:
def payoff(debt: float, N: int, interest: float, tax: float = 0.19):
    q = 1 + interest / 12 * (1-tax)
    return debt * q**N * (1 - q) / (1 - q**N)

payoff(debt=100_000, N=120, interest=0.15)

1443.3941075099476

In [6]:
def payoff_print(debt: float, N: int, interest: float, tax: float = 0.19):
    return f'Pay off monthly: {payoff(debt, N, interest, tax):.2f}'
interact(payoff_print, debt=debt_text, N=months_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), IntSlider(value=1, descripti…

## Number of periods
We are interested in the number of periods $N$ we need to pay off the debt $D_0$, with $p\%$ interest rate, $s\%$ tax fee and $m=12$ frequency of compounding. Let's denote $q = 1 +r, \; r = \frac{p}{m} \cdot (1-s)$ To get formula for it, we need to know the amount of money we are willing to pay monthly. Using the same equation as before:

\begin{align}
    D_0 q^{nm} + c \cdot \frac{1 - q^N}{1-q} = 0 \\
    N = \log_q (c) - \log_q \left( -D_0(1-q) + c \right)
\end{align}

In [7]:
from math import log
def num_months(debt: float, payoff: float, interest: float, tax: float = 0.19):
    q = 1 + interest / 12 * (1-tax)
    return log(payoff, q) - log(debt * (1-q) + payoff, q)

num_months(debt=100_000, payoff=1443.394, interest=0.15)

120.00001737331263

In [8]:
def num_months_print(debt: float, payoff: float, interest: float, tax: float = 0.19):
    return f'Months : {num_months(debt, payoff, interest, tax):.2f}'
interact(num_months_print, debt=debt_text, payoff=payoff_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), FloatSlider(value=1000.0, de…

## Debt able to pay
We are interested in the debt $D_0$ to take, with $N$ periods, $p\%$ interest rate, $s\%$ tax fee and $m=12$ frequency of compounding. Let's denote $q = 1 +r, \; r = \frac{p}{m} \cdot (1-s)$ To get formula for it, we need to know the amount of money we are willing to pay monthly. Using the same equation as before:

\begin{equation}
    D_0 = \frac{-c (1-q^N)}{q^N (1-q)}.
\end{equation}

In [9]:
def debt_to_take(N: int, payoff: float, interest: float, tax: float = 0.19):
    q = 1 + interest / 12 * (1-tax)
    return payoff * (1-q**N) / (1-q) / q**N

def debt_to_take_print(N: int, payoff: float, interest: float, tax: float = 0.19):
    return f'Investment: {N*payoff}, debt : {debt_to_take(N, payoff, interest, tax):.2f}'
interact(debt_to_take_print, N=months_slider, payoff=payoff_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(IntSlider(value=1, description='Months:', max=360, min=1), FloatSlider(value=1000.0, des…

## Mortgage plot

In [10]:
def plot_debt(d_0: float, n: int, payoff: float, interest: float, tax: float = 0.19):
    """Plot debt amount as a function of periods for different compounding frequencies."""
    N = np.arange(1, n)
    d_n = debt(-d_0, N, payoff, interest, tax)
    diff = np.append(d_0 + d_n[0], np.diff(d_n))
    interests = payoff-diff
    
    plt.plot(N, -d_n)
    plt.plot(N, diff.cumsum())
    plt.plot(N, interests.cumsum())
    plt.plot(N, payoff*N)
    plt.xlabel('months')
    plt.ylabel('debt')
    plt.title(f'Debt {d_0:,.0f}, months: {n:n}, payment: {payoff:,.2f}')
    plt.legend(labels=['Balance', 'Total principal', f'Total interest {interests.sum():,.0f}', f'Total cost {payoff*n:,.0f}'])
    plt.show()

def plot_debt_periods(d_0: float, n: int, interest: float, tax: float = 0.19):
    pay = payoff(d_0, n, interest, tax)
    plot_debt(d_0, n, pay, interest, tax)

interact(plot_debt_periods, d_0=debt_text, n=months_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), IntSlider(value=1, descripti…

In [11]:
def plot_debt_payoff(d_0: float, payoff: int, interest: float, tax: float = 0.19):
    n = num_months(d_0, payoff, interest, tax)
    plot_debt(d_0, n, payoff, interest, tax)

interact(plot_debt_payoff, d_0=debt_text, payoff=payoff_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), FloatSlider(value=1000.0, de…

## Saving VS Mortgage
We know that with debt we need to pay additional iterests, question is, when is it more profitable to take debt instead of saving?

In [12]:
def interests(N: int, payoff: float, interest: float, tax: float = 0.19):
    return N*payoff - debt_to_take(N, payoff, interest, tax)

def saving_vs_mortgage(amount_to_save: float, monthly_saving: int, rent: float,
                       utilities: float, interest: float, tax: float = 0.19):
    N = np.ceil(amount_to_save / monthly_saving)
    return N*rent - interests(N, rent*utilities+monthly_saving, interest, tax)

def saving_vs_mortgage_print(amount_to_save: float, monthly_saving: int, rent: float, utilities: float, interest: float, tax: float = 0.19):
    N = np.ceil(amount_to_save / monthly_saving)
    return f'Rent: {N*rent}, Difference : {saving_vs_mortgage(amount_to_save, monthly_saving, rent, interest, tax):.2f}'

interact(saving_vs_mortgage_print, amount_to_save=save_text, monthly_saving=monthly_save_text, rent=rent_slider, utilities=utilities_slider,
         interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Amount to save:', step=100.0, style=DescriptionSt…

In [13]:
def plot_savings(amount_to_save: float, rent: float, utilities: float, interest: float, tax: float = 0.19):
    """Plot debt amount as a function of periods for different compounding frequencies."""
    monthly_savings = np.linspace(start=1_000, stop=10_000, num=1000)

    plt.plot(monthly_savings, saving_vs_mortgage(amount_to_save, monthly_savings, rent, utilities, interest, tax))
    plt.xlabel('monthly savings')
    plt.ylabel('Profit')
    #plt.title(f'Debt {d_0:,.0f}, months: {n:n}, payment: {payoff:,.2f}')
    #plt.legend(labels=['Balance', 'Total principal', f'Total interest {interests.sum():,.0f}', f'Total cost {payoff*n:,.0f}'])
    plt.show()

interact(plot_savings, amount_to_save=save_text, rent=rent_slider, utilities=utilities_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Amount to save:', step=100.0, style=DescriptionSt…